Pobieramy potrzebne biblioteki

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download(["punkt", "stopwords", "wordnet", "omw-1.4"])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\grzeg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\grzeg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\grzeg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\grzeg\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Dodajemy wyrazy, które nie zmieniają sensu wypowiedzi jeśli je usuniemy.

In [4]:
stopwords = nltk.corpus.stopwords.words("english")
stopwords.append("'s")
stopwords.append("'re")
stopwords.append(",")
stopwords.append(".")
stopwords.append("``")
stopwords.append("''")

Otwieramy nasze artykuły i dokonujemy lematyzacji (usuwamy odmiany słów)

In [5]:
f_1 = open('Johanessburg.txt').read()
f_2 = open('Canada.txt').read()
f_3 = open('Kfc.txt').read()

In [6]:
lemmatized = []

article = f_1
tokenized_article_word = word_tokenize(article)
filtered_1=[]
for w in tokenized_article_word:
    if w not in stopwords:
        filtered_1.append(w)

lemmatized.append(' '.join(filtered_1))

article = f_2
tokenized_article_word = word_tokenize(article)
filtered_2=[]
for w in tokenized_article_word:
    if w not in stopwords:
        filtered_2.append(w)

lemmatized.append(' '.join(filtered_2))

article = f_3
tokenized_article_word = word_tokenize(article)
filtered_3=[]
for w in tokenized_article_word:
    if w not in stopwords:
        filtered_3.append(w)

lemmatized.append(' '.join(filtered_3))

Zapisujemy bazę dokumentów jako wektory w macierzy

Macierz DTM oznacza macierz bazy dokumentów, w której zliczane jest każde słowo w dokumencie. Wszystkie zliczenia są zapisywane w wektorze. Mając trzy artykuły otrzymujemy macierz z trzech wektorów.

In [7]:
corpus = lemmatized
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

X_array = X.toarray()
print('Macierz DTM:\n', X_array)

Macierz DTM:
 [[2 1 1 ... 1 0 0]
 [2 0 0 ... 0 1 0]
 [0 1 0 ... 0 0 1]]


Tworzymy macierz TFIDF która pokazuje występowanie słowa podzielone przez liczbę wszystkich słów danego dokumentu pomnożone przez logarytm z liczby wszystkich dokumentów podzielonych przez liczbę dokumentów z tym słowem

In [8]:
dt = pd.DataFrame(X_array)
dt.columns = vectorizer.get_feature_names_out()

dt.iloc[0, :] = dt.iloc[0, :] / np.count_nonzero(X_array[0])
dt.iloc[1, :] = dt.iloc[1, :] / np.count_nonzero(X_array[1])
dt.iloc[2, :] = dt.iloc[2, :] / np.count_nonzero(X_array[2])

text_tf = TfidfVectorizer().fit_transform(corpus)

tfidf = pd.DataFrame(text_tf.toarray())
tfidf.columns = vectorizer.get_feature_names_out()
print('Macierz TFIDF:\n', tfidf)

Macierz TFIDF:
         000        10        11     125th        13        15        18  \
0  0.036194  0.018097  0.023795  0.023795  0.000000  0.000000  0.000000   
1  0.091744  0.000000  0.000000  0.000000  0.060316  0.060316  0.000000   
2  0.000000  0.032598  0.000000  0.000000  0.000000  0.000000  0.042862   

       1886     1970s      1974  ...     wraps      year     years      york  \
0  0.023795  0.023795  0.023795  ...  0.000000  0.072388  0.014054  0.023795   
1  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.035624  0.000000   
2  0.000000  0.000000  0.000000  ...  0.042862  0.097793  0.025315  0.000000   

       your     zoned       zoo      zulu     âł403       âł5  
0  0.023795  0.023795  0.000000  0.023795  0.000000  0.000000  
1  0.000000  0.000000  0.060316  0.000000  0.060316  0.000000  
2  0.000000  0.000000  0.000000  0.000000  0.000000  0.042862  

[3 rows x 832 columns]


Obliczamy prawdopodobieństwo cosinusowe między artykułami

In [10]:
p1 = dot(dt.iloc[0, :], dt.iloc[1, :])/(norm(dt.iloc[0, :])*norm(dt.iloc[1, :]))
print('Podobieństwo cosinusowe artykułów Johanessburg i Canada:', p1)
p2 = dot(dt.iloc[0, :], dt.iloc[2, :])/(norm(dt.iloc[0, :])*norm(dt.iloc[2, :]))
print('Podobieństwo cosinusowe artykułów Johanessburg i KFC:', p2)
p3 = dot(dt.iloc[1, :], dt.iloc[2, :])/(norm(dt.iloc[1, :])*norm(dt.iloc[2, :]))
print('Podobieństwo cosinusowe artykułów Canada i KFC:', p3)

Podobieństwo cosinusowe artykułów Johanessburg i Canada: 0.0580618448093488
Podobieństwo cosinusowe artykułów Johanessburg i KFC: 0.11151437624878197
Podobieństwo cosinusowe artykułów Canada i KFC: 0.04362652574218082


Wnioski: Najbardziej zbliżone do siebie są artykuły Johanessburg i KFC